<a href="https://colab.research.google.com/github/ernerdgn/Pusula_Eren_Erdogan/blob/main/Pusula_DSC_Refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get data set
#https://github.com/ernerdgn/Pusula_Eren_Erdogan/raw/refs/heads/main/Talent_Academy_Case_DT_2025.xlsx

github_raw_url = 'https://github.com/ernerdgn/Pusula_Eren_Erdogan/raw/refs/heads/main/Talent_Academy_Case_DT_2025.xlsx'

!wget -O 'Talent_Academy_Case_DT_2025.xlsx' {github_raw_url}

--2025-09-05 13:31:43--  https://github.com/ernerdgn/Pusula_Eren_Erdogan/raw/refs/heads/main/Talent_Academy_Case_DT_2025.xlsx
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ernerdgn/Pusula_Eren_Erdogan/refs/heads/main/Talent_Academy_Case_DT_2025.xlsx [following]
--2025-09-05 13:31:44--  https://raw.githubusercontent.com/ernerdgn/Pusula_Eren_Erdogan/refs/heads/main/Talent_Academy_Case_DT_2025.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132900 (130K) [application/octet-stream]
Saving to: ‘Talent_Academy_Case_DT_2025.xlsx’

Talent_Academy_Case 100%[===================>] 129.79K  --.-KB/s 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
# load data, initial cleaning, duplicate handling
def load_and_clean_initial(file_path):
  df = pd.read_excel(file_path)
  print(f"original shape: {df.shape}")

  df.drop_duplicates(inplace=True)
  print(f"shape after dropping duplicates: {df.shape}")

  df['TedaviSuresi_sayisal'] = pd.to_numeric(df['TedaviSuresi'].str.extract('(\\d+)', expand=False))
  df['UygulamaSuresi_sayisal'] = pd.to_numeric(df['UygulamaSuresi'].str.extract('(\\d+)', expand=False))

  for col in ['KronikHastalik', 'Alerji', 'Tanilar', 'TedaviAdi', 'UygulamaYerleri']:
      df[f'{col}_cleaned'] = df[col].str.lower().str.replace('i̇', 'i')

  df['KronikHastalik_cleaned'] = df['KronikHastalik_cleaned'].str.replace('hiportiroidizm', 'hipotirodizm')
  df['Alerji_cleaned'] = df['Alerji_cleaned'].str.replace('volteren', 'voltaren')
  df['TedaviAdi_cleaned'] = df['TedaviAdi_cleaned'].str.replace(',', '').str.strip()

  print("initial cleaning complete!\n")
  return df

In [ ]:
# preprocessing, missing values handler
def handle_missing_values(df):
  df['Alerji_cleaned'].fillna('yok', inplace=True)
  df['KronikHastalik_cleaned'].fillna('yok', inplace=True)
  df['Tanilar_cleaned'].fillna(df['Tanilar_cleaned'].mode()[0], inplace=True)
  df['UygulamaYerleri_cleaned'].fillna(df['UygulamaYerleri_cleaned'].mode()[0], inplace=True)
  df['Cinsiyet'].fillna(df['Cinsiyet'].mode()[0], inplace=True)
  df['Bolum'].fillna(df['Bolum'].mode()[0], inplace=True)
  df['KanGrubu'].fillna('bilinmiyor', inplace=True)
  print("no more missing values!\n")
  return df

In [ ]:
# outlier capping
def handle_outliers(df):
  numerical_features = ['Yas', 'UygulamaSuresi_sayisal']
  for col in numerical_features:
    Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(upper=upper_bound)
  print("CAP CAP!\n")
  return df

In [ ]:
# feature engineering, creating new features for the columns
def engineer_features(df):
  # count features
  df['Num_Uygulama_Yerleri'] = df['UygulamaYerleri_cleaned'].apply(lambda x: len(x.split(',')))
  df['Num_Kronik_Hastalik'] = df['KronikHastalik_cleaned'].apply(lambda x: 0 if x == 'yok' else len(x.split(',')))
  df['Num_Alerji'] = df['Alerji_cleaned'].apply(lambda x: 0 if x == 'yok' else len(x.split(',')))
  df['Num_Tanilar'] = df['Tanilar_cleaned'].apply(lambda x: 0 if x in ['yok', ''] else len(x.split(',')))

  # "has_" columns
  all_diagnoses_counts = df['Tanilar_cleaned'].dropna().str.split(',').explode().str.strip().value_counts()
  unwanted_items = ['', 'şimdiki']
  filtered_diagnoses_counts = all_diagnoses_counts.drop(labels=unwanted_items, errors='ignore')
  top_15_diagnoses = filtered_diagnoses_counts.nlargest(15)

  for diagnosis in top_15_diagnoses.index:
    col_name = "has_" + diagnosis.replace(' ', '_').replace('[', '').replace(']', '').replace('.', '')
    df[col_name] = df['Tanilar_cleaned'].apply(lambda x: 1 if diagnosis in x else 0)

  # age groups
  df['Yas_Group'] = pd.cut(df['Yas'], bins=[0, 18, 36, 56, 100], labels=['Adolescent', 'Young Adult', 'Adult', 'Senior'], right=False)

  # target val classes for classification
  def create_target_class(sessions):
    if sessions == 15.0: return '15_Seans'
    elif sessions == 10.0: return '10_Seans'
    elif sessions == 30.0: return '30_Seans'
    else: return 'Other'
  df['TedaviSuresi_Class'] = df['TedaviSuresi_sayisal'].apply(create_target_class)

  print("feature engineering done!\n")
  return df

In [ ]:
# transformation and sclaing, OHE, target encoding and num scaling
def encode_and_scale(df):

  # LabelEncoder for target variable
  le = LabelEncoder()
  df['TedaviSuresi_Encoded'] = le.fit_transform(df['TedaviSuresi_Class'])

  # OHE for tedaviAdi
  top_tedavi = df['TedaviAdi_cleaned'].value_counts().nlargest(15).index
  df['TedaviAdi_Top15'] = df['TedaviAdi_cleaned'].apply(lambda x: x if x in top_tedavi else 'Other')
  df = pd.get_dummies(df, columns=['TedaviAdi_Top15'], prefix='TedaviAdi')

  # OHE for categoricals
  categorical_cols = ['Cinsiyet', 'KanGrubu', 'Uyruk', 'Bolum', 'Yas_Group']
  df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

  # dropping all unnec. columns
  cols_to_drop = [
    'Yas', 'TedaviSuresi', 'UygulamaSuresi', 'KronikHastalik', 'Alerji',
    'Tanilar', 'TedaviAdi', 'UygulamaYerleri', 'KronikHastalik_cleaned',
    'Alerji_cleaned', 'Tanilar_cleaned', 'TedaviAdi_cleaned', 'UygulamaYerleri_cleaned',
    'TedaviSuresi_sayisal', # target leakage
    'TedaviSuresi_Class'    # encoded
  ]
  df.drop(columns=cols_to_drop, inplace=True)

  # scaling
  scaler = StandardScaler()
  cols_to_scale = [
    'UygulamaSuresi_sayisal', 'Num_Uygulama_Yerleri', 'Num_Kronik_Hastalik',
    'Num_Alerji', 'Num_Tanilar'
  ]
  df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

  print("encoding and scaling done!\n")
  return df

In [ ]:
# main()
if __name__ == '__main__':
  FILE_PATH = "Talent_Academy_Case_DT_2025.xlsx"

  df_processed = load_and_clean_initial(FILE_PATH)
  df_processed = handle_missing_values(df_processed)
  df_processed = handle_outliers(df_processed)
  df_processed = engineer_features(df_processed)
  df_final = encode_and_scale(df_processed)

  print("=== model_ready_data ===")
  print(df_final.info())

  # export
  df_final.to_excel("model_ready_data_REFACTORED.xlsx", index=False)

original shape: (2235, 13)
shape after dropping duplicates: (1307, 13)
initial cleaning complete!

no more missing values!

CAP CAP!

feature engineering done!

encoding and scaling done!

=== model_ready_data ===
<class 'pandas.core.frame.DataFrame'>
Index: 1307 entries, 0 to 2234
Data columns (total 63 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   HastaNo                                    1307 non-null   int64  
 1   UygulamaSuresi_sayisal                     1307 non-null   float64
 2   Num_Uygulama_Yerleri                       1307 non-null   float64
 3   Num_Kronik_Hastalik                        1307 non-null   float64
 4   Num_Alerji                                 1307 non-null   float64
 5   Num_Tanilar                                1307 non-null   float64
 6   has_dorsalji                               1307 non-null   int64  
 7   has_diğer                      

/tmp/ipython-input-448428369.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Alerji_cleaned'].fillna('yok', inplace=True)
/tmp/ipython-input-448428369.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

====================================================================